In [3]:
!pip install tensorflow

In [4]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [5]:
# 1. Data Preprocessing

# Load the dataset
data = pd.read_csv(r"C:\Users\rohit\Downloads\archive\Reddit_Data.csv")

In [6]:
# Drop rows where 'clean_comment' is NaN
data = data.dropna(subset=['clean_comment'])

In [7]:
# Tokenize the "clean_comment" column
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(data['clean_comment'])
sequences = tokenizer.texts_to_sequences(data['clean_comment'])

In [8]:
# Pad the sequences
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

In [9]:
# Convert categories to one-hot encoding
labels = to_categorical(data['category'] + 1)  # Adding 1 to make categories 0, 1, and 2

In [10]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

In [11]:
# 2. Model Building

model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=100),
    LSTM(64, return_sequences=True),
    Dropout(0.5),
    LSTM(64),
    Dense(3, activation='softmax')
])

In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
# 3. Training

model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/5
929/929 [==============================] - 81s 84ms/step - loss: 1.0410 - accuracy: 0.4545 - val_loss: 1.0619 - val_accuracy: 0.4279
Epoch 2/5
929/929 [==============================] - 81s 87ms/step - loss: 1.0032 - accuracy: 0.4874 - val_loss: 0.7177 - val_accuracy: 0.6925
Epoch 3/5
929/929 [==============================] - 79s 85ms/step - loss: 0.5099 - accuracy: 0.7999 - val_loss: 0.3460 - val_accuracy: 0.8805
Epoch 4/5
929/929 [==============================] - 80s 86ms/step - loss: 0.2826 - accuracy: 0.9100 - val_loss: 0.2816 - val_accuracy: 0.9089
Epoch 5/5
929/929 [==============================] - 79s 85ms/step - loss: 0.2200 - accuracy: 0.9327 - val_loss: 0.3162 - val_accuracy: 0.9096


In [14]:
# 4. Evaluation

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")

233/233 [==============================] - 5s 19ms/step - loss: 0.3162 - accuracy: 0.9096
Test Accuracy: 90.96%
